In [21]:
import numpy as np
import pandas as pd
from datetime import datetime

In [22]:
# 주중-일자ver 파일
df_st = pd.read_csv('data/20230601_SUBWAY_MONTH_주중_월ver.csv',encoding='utf-8', sep=',')
df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df_st.index:
    if df_st.loc[i, '지하철역'][-1] == '역':    # 서울역 -> 서울
        df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df_st.to_csv(f'data/{f_date}_SUBWAY_MONTH_주중_월ver.csv',index=False,encoding='utf-8') # 0601자 파일생성

In [23]:
df = pd.read_csv(f'data/{f_date}_SUBWAY_MONTH_주중_월ver.csv',encoding='utf-8', sep=',')
df.head()

,사용월,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,주중주말
0,202101,일산선,백석,2724,2825,5549,주중
1,202101,일산선,마두,1638,1652,3290,주중
2,202101,3호선,을지로3가,1024,1009,2033,주중
3,202101,3호선,종로3가,1255,1632,2887,주중
4,202101,3호선,안국,2934,3029,5963,주중


In [24]:
df.drop(columns=['주중주말'])

,사용월,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
0,202101,일산선,백석,2724,2825,5549
1,202101,일산선,마두,1638,1652,3290
2,202101,3호선,을지로3가,1024,1009,2033
3,202101,3호선,종로3가,1255,1632,2887
4,202101,3호선,안국,2934,3029,5963
...,...,...,...,...,...,...
156828,202112,6호선,월곡,10214,9924,20138
156829,202112,6호선,상월곡,5619,4842,10461
156830,202112,6호선,돌곶이,8265,7811,16076
156831,202112,6호선,석계,11189,11335,22524


In [5]:
df.호선명.nunique(), df.호선명.unique()

(25,
 array(['일산선', '3호선', '장항선', '중앙선', '4호선', '2호선', '1호선', '5호선', '6호선',
        '7호선', '수인선', '8호선', '9호선', '9호선2~3단계', '경강선', '경부선', '경원선', '경의선',
        '경인선', '경춘선', '공항철도 1호선', '과천선', '분당선', '안산선', '우이신설선'],
       dtype=object))

In [6]:
df.지하철역.nunique()

523

In [26]:
######### sub_info 파일 만들기 #############
# df_sub = df.to_csv('data//main//lines/주중/2021_주중_sub_info.csv')

In [25]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    frame = df[df['호선명']==line].copy()   # 어떤 역할?
    frame = frame[['사용월', '호선명', '지하철역', '승차총승객수', '하차총승객수', '승하차총승객수']]
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'
    frame.to_csv(f'data/main//temp_files/{line}.csv',index=False,encoding='utf-8')

In [10]:
####### 호선 merge (파일 merge)########
path = 'data/main/temp_files/'
line_info = [
    ([f'{path}1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
    ([f'{path}2호선.csv'], '2호선'),
    ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
    ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
    ([f'{path}5호선.csv'], '5호선'),
    ([f'{path}6호선.csv'], '6호선'),
    ([f'{path}7호선.csv'], '7호선'),
    ([f'{path}8호선.csv'], '8호선'),
    ([f'{path}9호선.csv', f'{path}9호선2~3단계.csv'], '9호선'),
    ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
    ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선'),
    ([f'{path}우이신설선.csv'], '우이신설선'),
    ([f'{path}경춘선.csv'], '경춘선'),
    ([f'{path}경강선.csv'], '경강선'),
    ([f'{path}공항철도 1호선.csv'], '공항철도')
]
line_list = []
for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    line_list.append(res)
    res.to_csv(f'data/main//lines/주중/{line_name}.csv', index=False)
final = pd.concat(line_list,axis=0)


In [11]:
final.to_csv('data/main//lines/주중/merged_lines.csv',index=False)   # // 두개의미?

```주말 파일들도 똑같은 식 수행```

In [27]:
# 주말-일ver 파일
df_st = pd.read_csv('data/20230601_SUBWAY_MONTH_주말_월ver.csv',encoding='utf-8', sep=',')
df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df_st.index:
    if df_st.loc[i, '지하철역'][-1] == '역':    # 서울역 -> 서울
        df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df_st.to_csv(f'data/{f_date}_SUBWAY_MONTH_주말_월ver.csv',index=False,encoding='utf-8') # 0601자 파일생성

In [28]:
df = pd.read_csv(f'data/{f_date}_SUBWAY_MONTH_주말_월ver.csv')
df.head()

,사용월,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수,주중주말
0,202101,9호선,노들,1719,1527,3246,주말
1,202101,우이신설선,삼양사거리,1112,1102,2214,주말
2,202101,우이신설선,삼양,917,1019,1936,주말
3,202101,우이신설선,화계,1470,1315,2785,주말
4,202101,우이신설선,가오리,1653,1625,3278,주말


In [29]:
df.drop(columns=['주중주말'])

,사용월,호선명,지하철역,승차총승객수,하차총승객수,승하차총승객수
0,202101,9호선,노들,1719,1527,3246
1,202101,우이신설선,삼양사거리,1112,1102,2214
2,202101,우이신설선,삼양,917,1019,1936
3,202101,우이신설선,화계,1470,1315,2785
4,202101,우이신설선,가오리,1653,1625,3278
...,...,...,...,...,...,...
62441,202112,8호선,암사,7207,7005,14212
62442,202112,7호선,부평구청,2769,2615,5384
62443,202112,7호선,굴포천,3769,3821,7590
62444,202112,7호선,삼산체육관,2072,2018,4090


In [5]:
df.호선명.nunique(), df.호선명.unique()

(25,
 array(['9호선', '우이신설선', '안산선', '중앙선', '장항선', '일산선', '경인선', '수인선', '분당선',
        '과천선', '공항철도 1호선', '경춘선', '경의선', '경원선', '경부선', '경강선', '9호선2~3단계',
        '8호선', '7호선', '6호선', '5호선', '4호선', '3호선', '2호선', '1호선'],
       dtype=object))

In [6]:
df.지하철역.nunique()

523

In [30]:
######### sub_info 파일 만들기 #############
df_sub = df.to_csv('data//main//lines/주말/2021_주말_sub_info.csv')

In [18]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    frame = df[df['호선명']==line].copy()   # 어떤 역할?
    frame = frame[['사용월', '호선명', '지하철역', '승차총승객수', '하차총승객수', '승하차총승객수']]
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'
    frame.to_csv(f'data/main//temp_files/{line}.csv',index=False,encoding='utf-8')

In [19]:
####### 호선 merge (파일 merge)########
path = 'data/main/temp_files/'
line_info = [
    ([f'{path}1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
    ([f'{path}2호선.csv'], '2호선'),
    ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
    ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
    ([f'{path}5호선.csv'], '5호선'),
    ([f'{path}6호선.csv'], '6호선'),
    ([f'{path}7호선.csv'], '7호선'),
    ([f'{path}8호선.csv'], '8호선'),
    ([f'{path}9호선.csv', f'{path}9호선2~3단계.csv'], '9호선'),
    ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
    ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선'),
    ([f'{path}우이신설선.csv'], '우이신설선'),
    ([f'{path}경춘선.csv'], '경춘선'),
    ([f'{path}경강선.csv'], '경강선'),
    ([f'{path}공항철도 1호선.csv'], '공항철도')
]
line_list = []
for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    line_list.append(res)
    res.to_csv(f'data/main//lines/주말/{line_name}.csv', index=False)
final = pd.concat(line_list,axis=0)


In [20]:
final.to_csv('data/main//lines/주말/merged_lines.csv',index=False)   # // 두개의미?